In [ ]:
from os.path import exists
from urllib.request import urlretrieve
import geopandas as gpd
import hvplot.pandas

In [ ]:
SIZE = 400

In [ ]:
def gdf_from_wfs(layer):
    file = f'{layer}.json'
    url = f"https://data.wien.gv.at/daten/geo?service=WFS&request=GetFeature&version=1.1.0&typeName=ogdwien:{layer}&srsName=EPSG:4326&outputFormat=json"
    if not exists(file):
        urlretrieve(url, file)
    return gpd.read_file(file)

In [ ]:
gdf = gdf_from_wfs('ELADESTELLEOGD')

In [ ]:
gdf.hvplot(geo=True, tiles='OSM', width=SIZE, height=SIZE, hover_cols=['DESIGNATION']).opts(active_tools=['wheel_zoom'])

In [ ]:
def hvplot_with_buffer(gdf, buffer_size, title=''):
    buffered = gdf.to_crs('epsg:31287').buffer(100)
    buffered = gdf.copy().set_geometry(buffered).to_crs('epsg:4326')
    
    plot = ( buffered.hvplot(geo=True, title=title, tiles='OSM', width=SIZE, height=SIZE, alpha=0.5, line_width=0) * 
      gdf.hvplot(geo=True, hover_cols=['DESIGNATION']) 
    ).opts(active_tools=['wheel_zoom'])
    
    return plot

In [ ]:
hvplot_with_buffer(gdf, 100)

In [ ]:
gdf2 = gdf_from_wfs('CITYBIKEOGD')

In [ ]:
hvplot_with_buffer(gdf, 100, title='EV Charging Stations') + hvplot_with_buffer(gdf2, 100, title='Citybike Stations')